In [1]:
import numpy as np
import matplotlib
matplotlib.use('nbAgg')
import matplotlib.pyplot as plt

#load quad to check you result
from scipy.integrate import quad


In [2]:
class VEGAS:
    def __init__(self,Func,NBins,NSamplesPerBin,NRuns,alpha,K,_TINY=1.e-50,_SMOOTH=False):
        self.Func=Func
        self.NBins=NBins
        self.NSamplesPerBin=NSamplesPerBin
        
        
        self.NRuns=NRuns #number of runs. This refines the grid.
        self.alpha=alpha#the aplha parameter
        self.K=K #the K parameter
        self._TINY=_TINY#define a tiny number (if a weight is smaller than that, then set this weight to 1)
        self._SMOOTH=_SMOOTH#Choose if you want to first smooth the m_{i}'s (see comment above)

        
        
        #--initialize some of the variables for clarity--#
        self.BinSize=[1/float(NBins)  for i in range(NBins) ]
        self.GridPoints=[i/float(NBins) for i in range(NBins+1)]# (number of grid points)=(number of bins)+1
        
        
    def TotalInt(self):
        #calculate the integral of Func in [0,1]
        Integral=0
        for Bin in range(self.NBins):
            _min=self.GridPoints[Bin]
            _max=self.GridPoints[Bin+1]
            for Sample in self.Samples[Bin]:
                Integral+=self.Func(Sample)*self.BinSize[Bin]/float(self.NSamplesPerBin)

        return Integral
    
    def abs_TotalInt(self):
        #calculate the integral of |Func| in [0,1]
        Integral=0
        for Bin in range(self.NBins):
            _min=self.GridPoints[Bin]
            _max=self.GridPoints[Bin+1]
            for Sample in self.Samples[Bin]:
                Integral+=np.abs(self.Func(Sample))*self.BinSize[Bin]/float(self.NSamplesPerBin)

        return Integral
    
    
    def BinInt(self,Bin):
        #calculate the integral of Func in the i Bin
        Integral=0
        _min=self.GridPoints[Bin]
        _max=self.GridPoints[Bin+1]
        for Sample in self.Samples[Bin]:
            Integral+=self.Func(Sample)*self.BinSize[Bin]/float(self.NSamplesPerBin)

        return Integral

    def abs_BinInt(self,Bin):
        #calculate the integral of Func in the i Bin
        Integral=0
        _min=self.GridPoints[Bin]
        _max=self.GridPoints[Bin+1]
        for Sample in self.Samples[Bin]:
            Integral+=np.abs(self.Func(Sample))*self.BinSize[Bin]/float(self.NSamplesPerBin)

        return Integral
    
    
    
    
    def GenSamples(self):
        #Generate the samples
        #initialize Samples array with dimensions [NSamplesPerBin][NBins] (for simplicity)
        self.Samples=[[0 for i in range(self.NSamplesPerBin)] for j in range(self.NBins)]
        for Bin in range(self.NBins):
            _min=self.GridPoints[Bin]
            for Sample in range(self.NSamplesPerBin):
                #samples iin each bin (ie in [GridPoints[Bin],GridPoints[Bin+1]])
                self.Samples[Bin][Sample]=np.random.rand()*self.BinSize[Bin]+_min

    def CalcWeights(self):
        #calculate the m_i's (call them weights)
        self.Weights=[ 0 for i in range(self.NBins)]
        Total=self.abs_TotalInt()
        for Bin in range(self.NBins):
            self.Weights[Bin]=self.abs_BinInt(Bin)/Total#this is m_{i}/K in Lapage's paper.
        #I need he log of Weights[Bin]. But if it is small, then the log is not a good choice
        if self._SMOOTH:
            #smooth the weights
            self.Weights[0]=(self.Weights[0]+self.Weights[1])/2.
            self.Weights[-1]=(self.Weights[-1]+self.Weights[-2])/2.
            for Bin in range(1,self.NBins-1):
                self.Weights[Bin]=(self.Weights[Bin-1]+self.Weights[Bin]+self.Weights[Bin+1])/3.

        for Bin in range(self.NBins):
            if self.Weights[Bin]>self._TINY:
                self.Weights[Bin]=( (self.Weights[Bin]-1)/float(np.log(self.Weights[Bin])) )**self.alpha
            else:
                self.Weights[Bin]=self.Weights[Bin]**self.alpha#if Weights[Bin]<_TINY, then the log explodes. So it is better to se it this way

            self.Weights[Bin]=self.K*self.Weights[Bin]+1. #we should have at least one sub-bin for each bin!


    
    def Calculate(self):
        for Run in range(self.NRuns):
            self.GenSamples()
            self.CalcWeights()





        #initialize the new GridPoints
        NewGridPoints=[0 for i in self.GridPoints]
        NewBinSize=[0 for i in self.BinSize]


        #calculate the newBinSize[0], so that all entries in newBinSize add up to 1. 
        _tmp=0
        for j in range(self.NBins):
            _tmp+=self.BinSize[j]/self.Weights[j]
        NewBinSize[0]=_tmp**(-1)*self.BinSize[0]/self.Weights[0]

        #You can now calculate the others as well
        for i in range(1,self.NBins):
            NewBinSize[i]=NewBinSize[0]*(self.BinSize[i]/self.BinSize[0]*self.Weights[0]/self.Weights[i])


        #set the BinSize to be the NewBinSize
        del self.BinSize
        self.BinSize=NewBinSize[:]

        #find the new GridPoints from the new BinSize
        for i in range(1,self.NBins+1):
            NewGridPoints[i]=NewGridPoints[i-1]+self.BinSize[i-1]

        del self.GridPoints
        self.GridPoints=NewGridPoints[:]

In [16]:
def func(x):
    #return np.exp(-(x-0.3)**2/(2*0.1**2))
    return np.sin(x*10)*np.exp(-(x-0.3)**2/(2*0.05**2))


In [48]:
Int=VEGAS(func,NBins=50,NSamplesPerBin=50,NRuns=200,alpha=0.01,K=100.)

In [49]:
Int.Calculate()

In [50]:
Int.TotalInt()

0.015132258157632528

In [51]:
quad(func,0,1)[0]

0.015608519853793454